In [1]:
import tensorflow as tf

In [10]:
def input_fn():
    # repeat(step_repeat_number)
    dataset = tf.data.TextLineDataset('./up_down_dataset.csv')\
        .batch(20)\
        .repeat(9999)\
        .make_one_shot_iterator()\
        .get_next()
    lines = tf.decode_csv(dataset, record_defaults=[[0]]*11)
    feature = tf.stack(lines[:-1], axis=1)
    label = tf.expand_dims(lines[-1], axis=-1)
    
    feature = tf.cast(feature, tf.float32)
    label = tf.cast(label, tf.float32)
    return feature, label

In [13]:
def model_fn(features, labels, mode):  
    # estimator에서 원하는 모양이 세가지 파라미터를 똑같은 이름으로 받아야한다.
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    
    layer1 = tf.layers.dense(features, units=10, activation=tf.nn.relu) # units = the number of node
    layer2 = tf.layers.dense(layer1, units=10, activation=tf.nn.relu)
    layer3 = tf.layers.dense(layer2, units=10, activation=tf.nn.relu)
    layer4 = tf.layers.dense(layer3, units=10, activation=tf.nn.relu)
    out = tf.layers.dense(layer4, units=1)  # label = 0 or 1
    
    if TRAIN:
        gs = tf.train.get_global_step()
        # loss function
        loss = tf.losses.sigmoid_cross_entropy(labels, out)
        train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step=gs)
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,train_op=train_op)
        
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, out)
        # prediction
        pred = tf.nn.sigmoid(out)  # number of 0~1 
        # accuracy
        acc = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,eval_metric_ops={'acc':acc})    

In [16]:
if __name__ =='__main__':  # 이렇게 사용하는걸 적극 권장한다.
    
    # estimator 내부에서 생성되는 로그를 보기위한 것, jupyter에서 실행하면 바로나오더라.
    tf.logging.set_verbosity(tf.logging.INFO)  
    
    estimate = tf.estimator.Estimator(model_fn)
    estimate.train(input_fn, steps=1000)
    estimate.evaluate(input_fn,steps=10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpmk380tye', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4a3050ee80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())  # accuracy/count is local variable
    
    for i in range(1000):
        _, loss_, acc_ = sess.run([train_op, loss, acc])
        print('step: {}, loss: {}, accuracy: {}'.format(i, loss_, acc_))

step: 0, loss: 0.6767985820770264, accuracy: (0.0, 0.5)
step: 1, loss: 0.6123156547546387, accuracy: (0.5, 0.5)
step: 2, loss: 0.5722694396972656, accuracy: (0.5, 0.5)
step: 3, loss: 0.5380921959877014, accuracy: (0.5, 0.5)
step: 4, loss: 0.5072094202041626, accuracy: (0.5, 0.5)
step: 5, loss: 0.4885789453983307, accuracy: (0.5, 0.5)
step: 6, loss: 0.4722108840942383, accuracy: (0.5, 0.5)
step: 7, loss: 0.4563263952732086, accuracy: (0.5, 0.5)
step: 8, loss: 0.447798490524292, accuracy: (0.5, 0.5)
step: 9, loss: 0.4342973828315735, accuracy: (0.5, 0.5)
step: 10, loss: 0.4226199686527252, accuracy: (0.5, 0.54545456)
step: 11, loss: 0.41301020979881287, accuracy: (0.54545456, 0.58333331)
step: 12, loss: 0.40245747566223145, accuracy: (0.58333331, 0.61538464)
step: 13, loss: 0.397083580493927, accuracy: (0.61538464, 0.64285713)
step: 14, loss: 0.38378798961639404, accuracy: (0.64285713, 0.66666669)
step: 15, loss: 0.3729310631752014, accuracy: (0.66666669, 0.6875)
step: 16, loss: 0.363883

step: 237, loss: 0.005243467632681131, accuracy: (0.97890294, 0.97899157)
step: 238, loss: 0.00520747248083353, accuracy: (0.97899157, 0.97907948)
step: 239, loss: 0.005170801654458046, accuracy: (0.97907948, 0.97916669)
step: 240, loss: 0.005135903134942055, accuracy: (0.97916669, 0.97925311)
step: 241, loss: 0.0050988816656172276, accuracy: (0.97925311, 0.97933882)
step: 242, loss: 0.005064092110842466, accuracy: (0.97933882, 0.97942388)
step: 243, loss: 0.0050299568101763725, accuracy: (0.97942388, 0.97950822)
step: 244, loss: 0.004995984025299549, accuracy: (0.97950822, 0.97959185)
step: 245, loss: 0.004962156526744366, accuracy: (0.97959185, 0.97967482)
step: 246, loss: 0.004929127637296915, accuracy: (0.97967482, 0.97975707)
step: 247, loss: 0.004896440543234348, accuracy: (0.97975707, 0.97983873)
step: 248, loss: 0.004865655675530434, accuracy: (0.97983873, 0.97991967)
step: 249, loss: 0.004832994192838669, accuracy: (0.97991967, 0.98000002)
step: 250, loss: 0.004801517352461815

step: 424, loss: 0.0020972879137843847, accuracy: (0.98820752, 0.98823529)
step: 425, loss: 0.0020892638713121414, accuracy: (0.98823529, 0.98826289)
step: 426, loss: 0.0020822319202125072, accuracy: (0.98826289, 0.98829037)
step: 427, loss: 0.002074667951092124, accuracy: (0.98829037, 0.98831773)
step: 428, loss: 0.0020679007284343243, accuracy: (0.98831773, 0.98834497)
step: 429, loss: 0.0020604869350790977, accuracy: (0.98834497, 0.98837209)
step: 430, loss: 0.0020533925853669643, accuracy: (0.98837209, 0.98839909)
step: 431, loss: 0.0020461922977119684, accuracy: (0.98839909, 0.98842591)
step: 432, loss: 0.0020390059798955917, accuracy: (0.98842591, 0.98845267)
step: 433, loss: 0.002032165415585041, accuracy: (0.98845267, 0.98847926)
step: 434, loss: 0.002024666639044881, accuracy: (0.98847926, 0.98850572)
step: 435, loss: 0.0020184244494885206, accuracy: (0.98850572, 0.98853213)
step: 436, loss: 0.0020107529126107693, accuracy: (0.98853213, 0.98855835)
step: 437, loss: 0.002004136

step: 605, loss: 0.0012445047032088041, accuracy: (0.99173552, 0.99174917)
step: 606, loss: 0.0012417675461620092, accuracy: (0.99174917, 0.99176276)
step: 607, loss: 0.0012385996524244547, accuracy: (0.99176276, 0.99177629)
step: 608, loss: 0.0012357691302895546, accuracy: (0.99177629, 0.99178982)
step: 609, loss: 0.0012327770236879587, accuracy: (0.99178982, 0.99180329)
step: 610, loss: 0.0012300326488912106, accuracy: (0.99180329, 0.9918167)
step: 611, loss: 0.0012271327432245016, accuracy: (0.9918167, 0.99183005)
step: 612, loss: 0.0012242377270013094, accuracy: (0.99183005, 0.9918434)
step: 613, loss: 0.0012214151211082935, accuracy: (0.9918434, 0.99185669)
step: 614, loss: 0.001218460383825004, accuracy: (0.99185669, 0.99186993)
step: 615, loss: 0.0012157608289271593, accuracy: (0.99186993, 0.9918831)
step: 616, loss: 0.0012127631343901157, accuracy: (0.9918831, 0.99189627)
step: 617, loss: 0.0012101707980036736, accuracy: (0.99189627, 0.99190938)
step: 618, loss: 0.0012071391101

step: 783, loss: 0.0008642099564895034, accuracy: (0.99361432, 0.99362242)
step: 784, loss: 0.000862611923366785, accuracy: (0.99362242, 0.99363059)
step: 785, loss: 0.0008611609227955341, accuracy: (0.99363059, 0.99363869)
step: 786, loss: 0.0008595295948907733, accuracy: (0.99363869, 0.99364674)
step: 787, loss: 0.000858127255924046, accuracy: (0.99364674, 0.99365485)
step: 788, loss: 0.0008564909803681076, accuracy: (0.99365485, 0.99366289)
step: 789, loss: 0.0008550502243451774, accuracy: (0.99366289, 0.99367088)
step: 790, loss: 0.0008535337401553988, accuracy: (0.99367088, 0.99367887)
step: 791, loss: 0.0008520171977579594, accuracy: (0.99367887, 0.99368685)
step: 792, loss: 0.0008505379082635045, accuracy: (0.99368685, 0.99369484)
step: 793, loss: 0.000848989759106189, accuracy: (0.99369484, 0.99370277)
step: 794, loss: 0.0008475842769257724, accuracy: (0.99370277, 0.9937107)
step: 795, loss: 0.000845985661726445, accuracy: (0.9937107, 0.99371856)
step: 796, loss: 0.000844621681

step: 952, loss: 0.0006596672465093434, accuracy: (0.99474788, 0.99475342)
step: 953, loss: 0.0006587104871869087, accuracy: (0.99475342, 0.9947589)
step: 954, loss: 0.0006577832973562181, accuracy: (0.9947589, 0.99476439)
step: 955, loss: 0.0006568605895154178, accuracy: (0.99476439, 0.99476987)
step: 956, loss: 0.0006558864843100309, accuracy: (0.99476987, 0.99477535)
step: 957, loss: 0.0006549918325617909, accuracy: (0.99477535, 0.99478078)
step: 958, loss: 0.0006539954920299351, accuracy: (0.99478078, 0.99478626)
step: 959, loss: 0.0006530735990963876, accuracy: (0.99478626, 0.99479169)
step: 960, loss: 0.000652139657177031, accuracy: (0.99479169, 0.99479711)
step: 961, loss: 0.0006512507097795606, accuracy: (0.99479711, 0.99480247)
step: 962, loss: 0.0006503163604065776, accuracy: (0.99480247, 0.9948079)
step: 963, loss: 0.0006493788678199053, accuracy: (0.9948079, 0.99481326)
step: 964, loss: 0.0006484846817329526, accuracy: (0.99481326, 0.99481863)
step: 965, loss: 0.00064752832